In [ ]:
%load_ext autoreload
%autoreload 2

import os
%matplotlib inline
import pickle
import pandas as pd

""" load data """
with open(os.path.join('..', 'dataset', 'experiments.pkl'), "rb") as experiments_file:
    experiments = pickle.load(experiments_file)

# display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100000)


# PID and MiCS Metal Oxide Semiconductor Sensor Timeseries

Use the interactive elements to inspect the sensor data compensation steps,
at various stages of preprocessing.

*This notebook needs an interactive kernel + access to the dataset in full. To run it, please clone the repository, acquire the dataset as described in the README and run it locally.*


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Define the figure and axis
fig, ax = plt.subplots(figsize=(10, 4.5))

# Define the interactive elements
but_experiment_class = widgets.ToggleButtons(
    options = ["Sampling_Experiments", "Fly-Through_Experiments", "Purging_Runs"],
    description='Select Experiment Class:',
    style={"button_width": "auto"},
)
experiments_options = dict()
for exp_class in experiments.keys():
    experiments_options[exp_class] = [f"E {i+1}" for i in range(len(experiments[exp_class]))]

but_experiment = widgets.ToggleButtons(
    options = experiments_options["Sampling_Experiments"],
    description='Select Experiment:',
    style={"button_width": "auto"},
)
but_sensor_type = widgets.ToggleButtons(
    options=['MiCS5524', 'MiCS6814_NH3', 'MiCS6814_NO2', 'PID-sensor'],
    description='Select Sensor Type:',
    style={"button_width": "auto"},
)

but_series = widgets.ToggleButtons(
    options=[('Voltage (raw)', 'voltage'), ('Voltage (sanitized, PID)', 'voltage_sanitized'), ('Resistance', 'resistance'), ('Resistance due to T/H', 'R_gas-free'), ('Rs/R0', 'Rs/R0'), ('ppm', 'ppm'), ('ppm Ethanol (PID)', 'ppm_ethanol'), ('ppm PG (PID)', 'ppm_PG'), ('ppm above background', 'ppm_relative')],
    description='Select a Time Series:',
    style={"button_width": "auto"},
)
slider_ma = widgets.IntSlider(
    value=60,
    min=0,
    max=300,
    step=1,
    description='Moving Average [s]:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'},  # Ensures the label is fully visible
    layout=widgets.Layout(width='800px')  # Makes the slider wider
)

# Function to update plot
def update_plot(*args):
    ax.clear()

    # extract values from interactive controls
    exp_class = but_experiment_class.value

    but_experiment.options = experiments_options[exp_class]

    i = int(but_experiment.value.split()[1]) - 1
    experiment = experiments[exp_class][i]
    sensor_type = but_sensor_type.value
    series = but_series.value
    series_label = {id_: label for label, id_ in but_series.options}[series]
    rolling_window = slider_ma.value

    # render plot
    ax.grid()
    channels = [('layers', i) for i in range(4)] + [('upstream', i) for i in [0, 1]]

    for key, idx in channels:
        try:
            data = experiment[key][idx][sensor_type]
            data = data.rolling(f'{rolling_window}s').mean()
            ax.plot(data[series], label=f"{key} {idx}")
        except KeyError:
            continue

    ax.tick_params(axis="x", labelrotation=45)
    ax.set_title(f"{sensor_type}: {experiment['path']}")
    ax.set_xlabel("Time")
    ax.set_ylabel(series_label)
    ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
    plt.tight_layout()

but_experiment_class.observe(update_plot, names='value')
but_experiment.observe(update_plot, names='value')
but_sensor_type.observe(update_plot, names='value')
but_series.observe(update_plot, names='value')
slider_ma.observe(update_plot, names='value')

# Display widgets and plot
display(but_experiment_class, but_experiment, but_sensor_type, but_series, slider_ma)
update_plot()

plt.show()

## Description

**Experiment Class** and **Experiment** select which timeseries is shown.

**Sensor Type** selects one of the four gas sensor channels. Only data for one of these sensors is shown at a time.

**Time Series** selects either the raw voltage signal, or one of the derived timeseries. Resistances are only available for the MOS senosrs ("MiCS"), and Ethanol/PG equivalents are only available for the PID sensor.

**Moving Average** allows you to smooth the data. Strong filtering makes it easier to see the general trends in the signal, while weak or no filtering show the fine peaks and noisiness of the sensors. Note that strong smoothing flattens the amplitude of the peaks significantly.

Matplotlib should expose some zoom controls in the user interface, that allow you to zoom into the timeseries to inspect them in detail.
